# exercise with pandas

every "..." should be replaced with a line of code

In [1]:
import requests
import zipfile
import io
import pandas as pd
import os
#declare parameters
puidata = os.getenv("PUIDATA")
if puidata is None:
    os.environ["PUIDATA"] = "%s/PUIdata"%os.getenv("HOME")
    puidata = os.getenv("PUIDATA")

Read what the PLUTO data is [here](https://www.wired.com/2013/08/nyc-pluto-data-map-party/)

Find the PLUTO dataset online, find the cvs file data (not the shape file), it should be about 46M

Download it as a zip file, unpack it in PUIDATAEach borough will be in a different file. The goal is to find land plot categories of which there is a higher number in Manhattan (MN) than in Brooklyn (BK)

Read in csv data as pandas dataframes (with pd.read_csv() )

In a Pandas DataFrame each column is a _series_ (a pandas datatype) to manipulate the data here you will mostly use methods (functions that are defined for a specific datatype and invoked with the _object.method()_ syntax) of the DataFrames (e.g. df.groupby() ) or of the series therein (e.g. series.astype() )

In [4]:
url="https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_18v1.zip" 
r = requests.get(url, stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(puidata)

In [6]:
mann = pd.read_csv(puidata+"/PLUTO_for_WEB/MN_18v1.csv")
brook = pd.read_csv(puidata+"/PLUTO_for_WEB/BK_18v1.csv")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (20,23,24,26,64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,20,22,23,24,25,26,64,65,80) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# read in te Manhattan and Brooklyn data in 2 separate dataframes w pandas

## show the first 10 lines of each df

In [50]:
# show manhattan 10 rows
mann.head(10)

,ZipCode,LandUse
0,10004.0,8.0
1,10004.0,8.0
2,10004.0,NaN
3,10004.0,7.0
4,10004.0,7.0
5,10004.0,7.0
6,10004.0,7.0
7,10004.0,7.0
8,10004.0,7.0
9,10004.0,9.0


In [51]:
# show brooklyn 10 rows
brook.head(10)

,ZipCode,LandUse
0,11201.0,11.0
1,11201.0,5.0
2,11201.0,4.0
3,11201.0,7.0
4,11201.0,7.0
5,NaN,NaN
6,11201.0,7.0
7,11201.0,7.0
8,11201.0,7.0
9,11201.0,7.0


## display all columns in each file

In [52]:
# show bk
brook.columns

Index(['ZipCode', 'LandUse'], dtype='object')

In [53]:
# show mn
mann.columns

Index(['ZipCode', 'LandUse'], dtype='object')

# drop all columns but ZipCode and LandUse

In [54]:
# drop columns here. Use the .drop method of pandas dataframes, i.e. something like bk.drop()... 
# make sure you drop "in place" and that you drop along the right axis - i.e. the columns. 
# you can use bk.drop? to the help with this function 
drop_cols=[x for x in mann.columns.get_values() if x not in ['ZipCode','LandUse']]
mann.drop(drop_cols,axis=1,inplace=True)
mann.head()

,ZipCode,LandUse
0,10004.0,8.0
1,10004.0,8.0
2,10004.0,NaN
3,10004.0,7.0
4,10004.0,7.0


In [55]:
# drop columns here...
drop_cols=[x for x in brook.columns.get_values() if x not in ['ZipCode','LandUse']]
brook.drop(drop_cols,axis=1,inplace=True)
brook.head()

,ZipCode,LandUse
0,11201.0,11.0
1,11201.0,5.0
2,11201.0,4.0
3,11201.0,7.0
4,11201.0,7.0


# use groupby to count the building per LandUse code 

In [56]:
# group Manhattan by land use. Make sure you understand how to use the groupby() method:
# this method returns a group object. those objects have methods and some of these methods return dataframes. 
# What do you want to do? you want to count all objects per each land use type. 
# So you want to use .groupby() to group by LandUse and then .count() to get the number of element per LandUse
# Other methods include .mean() to get the mean of a group, .std(), 
# and a generic method .apply() to which you can pass any function that operates on a column.
# All these methods return dataframes 
# (hint: use also reset_index) for each df
mngrouped = mann.groupby("LandUse").count().reset_index()
mngrouped

,LandUse,ZipCode
0,1.0,4019
1,2.0,16246
2,3.0,4881
3,4.0,7238
4,5.0,4465
5,6.0,726
6,7.0,386
7,8.0,2456
8,9.0,434
9,10.0,679


In [57]:
# group BK by land use
bkgrouped = brook.groupby("LandUse").count().reset_index()
bkgrouped

,LandUse,ZipCode
0,1.0,156271
1,2.0,66358
2,3.0,3437
3,4.0,22515
4,5.0,5743
5,6.0,4434
6,7.0,1040
7,8.0,4330
8,9.0,872
9,10.0,4229


# convert the landuse to integer 

In [58]:
# convert LandUse to int using the .astype() method of the series
mngrouped['LandUse'] = mngrouped.LandUse.astype(int)
bkgrouped['LandUse'] = bkgrouped.LandUse.astype(int)
bkgrouped.head()

,LandUse,ZipCode
0,1,156271
1,2,66358
2,3,3437
3,4,22515
4,5,5743


# merge the dataframes on land use 

In [59]:
# merge the dataframes on LandUse. You can use the pandas function merge ( _pd.merge)
both = mngrouped.merge(right=bkgrouped,on='LandUse',suffixes=('_x', '_y'))
both.columns

Index(['LandUse', 'ZipCode_x', 'ZipCode_y'], dtype='object')

# rename things so that you have descriptive names

## rename the columns with descriptive names. 


In [60]:
# Use the df.rename() method (hint: it takes a dictionary)
# now you can just call "both" instead of "both.head()" since the file should only have 11 columns
both.rename(columns={'ZipCode_x':'mn','ZipCode_y':'bk'},inplace=True)
both

,LandUse,mn,bk
0,1,4019,156271
1,2,16246,66358
2,3,4881,3437
3,4,7238,22515
4,5,4465,5743
5,6,726,4434
6,7,386,1040
7,8,2456,4330
8,9,434,872
9,10,679,4229


## find online the LandUse data dictionary and replace code w the verbose description of it


In [61]:
# set up Land Use values to descriptive tag
#taken from
lookup = {1:'One & Two Family Buildings',
2:'Multi-Family Walk-Up Buildings',
3:'Multi-Family Elevator Buildings',
4:'Mixed Residential & Commercial Buildings',
5:'Commercial & Office Buildings',
6:'Industrial & Manufacturing',
7:'Transportation & Utility',
8:'Public Facilities & Institutions',
9:'Open Space & Outdoor Recreation',
10:'Parking Facilities',
11:'Vacant Land'}

both['LandUse'] = both['LandUse'].map(lookup)

both

,LandUse,mn,bk
0,One & Two Family Buildings,4019,156271
1,Multi-Family Walk-Up Buildings,16246,66358
2,Multi-Family Elevator Buildings,4881,3437
3,Mixed Residential & Commercial Buildings,7238,22515
4,Commercial & Office Buildings,4465,5743
5,Industrial & Manufacturing,726,4434
6,Transportation & Utility,386,1040
7,Public Facilities & Institutions,2456,4330
8,Open Space & Outdoor Recreation,434,872
9,Parking Facilities,679,4229


# create a feature (a column, a series) with the ratio of number of buildings in BK vs MN and find out in which LandUse class Manhattan has more buildings than BK 

In [62]:
# calculate the ratio of BK to Man buildings per LandUse category
both['luration'] =  both.bk*1.0 / both.mn*1.0
both

,LandUse,mn,bk,luration
0,One & Two Family Buildings,4019,156271,38.883055
1,Multi-Family Walk-Up Buildings,16246,66358,4.084575
2,Multi-Family Elevator Buildings,4881,3437,0.704159
3,Mixed Residential & Commercial Buildings,7238,22515,3.110666
4,Commercial & Office Buildings,4465,5743,1.286226
5,Industrial & Manufacturing,726,4434,6.107438
6,Transportation & Utility,386,1040,2.694301
7,Public Facilities & Institutions,2456,4330,1.763029
8,Open Space & Outdoor Recreation,434,872,2.009217
9,Parking Facilities,679,4229,6.228277


In [64]:
# defined function that returns True if the ratio is larger than 0, False otherwise. 
# Use the series.apply() method to apply that function to the luratio series you jusrt created
both['Manhattan wins'] = both['luration'].apply(lambda x : True if x>0.0 else False)
both

,LandUse,mn,bk,luration,Manhattan wins
0,One & Two Family Buildings,4019,156271,38.883055,True
1,Multi-Family Walk-Up Buildings,16246,66358,4.084575,True
2,Multi-Family Elevator Buildings,4881,3437,0.704159,True
3,Mixed Residential & Commercial Buildings,7238,22515,3.110666,True
4,Commercial & Office Buildings,4465,5743,1.286226,True
5,Industrial & Manufacturing,726,4434,6.107438,True
6,Transportation & Utility,386,1040,2.694301,True
7,Public Facilities & Institutions,2456,4330,1.763029,True
8,Open Space & Outdoor Recreation,434,872,2.009217,True
9,Parking Facilities,679,4229,6.228277,True


# comment on what you found out. Does it make sense? 

###### considering the logic of the instruction, which is to check if the ratio is greater than 0... it doesn't make sense because there won't be a negative value thus all values will return as True. it's a different story if i were to use $0 < x < 1$, which may yield some results as False. But everything is True. 

###### However if i were to comment on the luration without considering the previous instruction, considering Brooklyn has a larger land area, the result shouldn't be too surprising that most of the ratios are above 1.  It's notable when we talk about multi-family elevator buildings, which is commonly seen in manhattan but not so much with Brooklyn because I heard these building tend to more expensive in construction, so if i were to investigate with socio-economic demographic features in mind, this can make sense. 